<a href="https://colab.research.google.com/github/samuel-zahner/projects/blob/master/Classify_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement
Fine-tune the ResNet 18 network to
classify the MNIST dataset. Report the confusion matrix, the accuracy, the f-score,
precision and recall of your classifier.

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
from google.colab import drive
from torchvision import datasets, models, transforms
import copy
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# TODO: Load pretrained model
device = torch.device('cuda')  # use gpu device
model = models.resnet18(pretrained=True)
model = model.to(device) #send model to gpu

# Number of classes in the dataset
num_classes = 10

# TODO: Update network to handle different number of output classes
model.fc = torch.nn.Linear(512, num_classes)

# TODO: Create an optimizer to update weights of the network
params_to_update = model.parameters()
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
#TODO: Load MNIST dataset(train & test set) 
#convert to dataloader to be able to iterate
train = torch.utils.data.DataLoader(datasets.MNIST(root='./data', train=True, download=True, transform = transforms.ToTensor()
),batch_size=64,shuffle=True)
test = torch.utils.data.DataLoader(datasets.MNIST(root='./data', train=False, download=True, transform = transforms.ToTensor()
),batch_size=64,shuffle=True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
#training model function 
def train_model(model, dataloaders, criterion, optimizer, num_epochs=2):

    #iterate through epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        running_loss = 0.0
        running_corrects = 0
        model.train() #model to train mode

        # Iterate through dataloaders 
        for inputs, labels in dataloaders:
              #print('in data loader')
              inputs = inputs.to(device)
              labels = labels.to(device)

              # zero param gradients
              optimizer.zero_grad()

              with torch.set_grad_enabled(True):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels.long())
                    _, preds = torch.max(outputs, 1)

                    loss.backward()
                    optimizer.step()

              # statistics
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders.dataset)
        epoch_acc = running_corrects.double() / len(dataloaders.dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format("train", epoch_loss, epoch_acc))

    return model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=1, padding=3,bias=False)
model.to(device)
# Train the model
model = train_model(model, train, criterion, optimizer, num_epochs=2)


Epoch 0/1
train Loss: 0.1722 Acc: 0.9488
Epoch 1/1
train Loss: 0.0395 Acc: 0.9880


In [ ]:
# TODO: Test network on test set of MNIST dataset
predicted = np.zeros((0))
actual = np.zeros((0))
model.eval() #set model to eval
model.to(device)

for input, labels in test:
  input = input.to(device)
  prediction = model(input) #get predictions 
  _,tensor = torch.max(prediction, 1)

  predicted = np.append(predicted, tensor.cpu().numpy())
  actual = np.append(actual, labels)

In [ ]:
# TODO: Display and analyze results
display_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#use sklearn to calculate confusion matrix and classification report (accuracy, precision, recall, fscore)
print (confusion_matrix(actual, predicted, labels=display_labels))
print (classification_report(actual, predicted, labels=display_labels))e

[[ 973    0    2    0    0    1    3    1    0    0]
 [   0 1123    1    1    2    1    4    3    0    0]
 [   1    0 1028    1    0    0    0    1    1    0]
 [   0    0    3  999    0    6    0    1    0    1]
 [   0    0    1    0  954    0    4    4    0   19]
 [   0    0    0    5    0  885    1    0    1    0]
 [   6    0    1    0    4    4  943    0    0    0]
 [   0    0    2    1    0    0    0 1024    1    0]
 [   2    1    5    5    1    5    0    2  950    3]
 [   0    2    0    1    5    2    0    6    2  991]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      0.99      0.99      1135
           2       0.99      1.00      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.97      0.98       982
           5       0.98      0.99      0.99       892
           6       0.99      0.98      0.99       958
           7       0.98      1.00   